# Generating Useful Wikidata Files

### Batch Invocation
Example batch command. The second argument is a notebook where the output will be stored. You can load it to see progress.

```
papermill Example7\ -\ Wikidata\ Outputs.ipynb example7.out.ipynb \
-p home /Users/pedroszekely/Downloads/kypher \
-p wiki_file all.10.tsv.gz \
-p output_folder output.all.10 \
-p temp_folder temp.all.10 \
-p delete_database true 
```

In [47]:
# Parameters
home = "/Users/pedroszekely/Downloads/kypher"
wiki_file = "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20200803-v3/all.tsv.gz"
wiki_file = "/Volumes/GoogleDrive/Shared drives/KGTK-public-graphs/wikidata-20200803-v3/all.tsv.gz"
label_file = "/Volumes/GoogleDrive/Shared drives/KGTK-public-graphs/wikidata-20200803-v3/part.label.en.tsv.gz"
output_path = "/Users/pedroszekely/Downloads/kypher"
cache_path = "/Users/pedroszekely/Downloads/kypher/temp.useful_wikidata_files_v3"
output_folder = "useful_wikidata_files_v3"
temp_folder = "temp.useful_wikidata_files_v3"
delete_database = "no"

In [2]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd

# from IPython.display import display, HTML, Image
# from pandas_profiling import ProfileReport

## Set up environment and folders to store the files

- `OUT` folder where the output files go
- `TEMP` folder to keep temporary files , including the database
- `kgtk` shortcut to invoke the kgtk software

The current implementation of some of the kgtk commands does not understand compressed files. In particular, `query` often rejects `gz` files.

To dos:

- Make sure that all files have id columns as `query` gets unhappy when files have no ids.
- Create an output folder for a subset of Wikidata without scholarly articles. This is half done: the remaining work is to subtract the scholarly articles from `EDGES` and repeat the workflow.
- Change the naming convention to make it clear which files are a partition of the original `EDGES`, so users know what files they need to get to have a full version.
- Create a qualifier file for the partition files of Wikidata: this is so that if a user gets one of the partitions, they can get the corresponding qualifier file.
- Add pagerank and other stats. We can compute the pagerank from the `all.item` file, so maybe should be called `all.item.pagerank.tsv`

Naming convention: the name `all` is redundant, we should consider removing it. I recomment using the prefix `part.` to name the partition of Wikidata, e.g., `part.label`, `part.quantity`. Files such as `P279` are not partitions as it is a subset of `part.item`.

If we create a subset of Wikidata, e.g., no scholarly articles, we could call it `minus.Q13442814`; if we remove galaxies too, we could call it `minus.Q13442814-Q318`, so the files would be `minus.Q13442814-Q318.part.quantity.tsv` (the idea of `all` is in contrast to `minus`). We can also have files that start with Qnodes, e.g, `Q5.part.quantity.tsv`; constructing such files is harder as we don't want dangling nodes in the item file.

In [48]:
os.environ['OUT'] = "{}/{}".format(output_path, output_folder)
os.environ['TEMP'] = "{}/{}".format(output_path, temp_folder)
os.environ['kgtk'] = "kgtk"
os.environ['kgtk'] = "time kgtk --debug"
os.environ['EDGES'] = wiki_file
os.environ['LABELS'] = label_file
if cache_path:
    os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cache_path)
else:
    os.environ['STORE'] = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)

In [49]:
!echo $OUT
!echo $TEMP
!echo $kgtk
!echo $EDGES
!echo $LABELS
!echo $STORE

/Users/pedroszekely/Downloads/kypher/useful_wikidata_files_v3
/Users/pedroszekely/Downloads/kypher/temp.useful_wikidata_files_v3
time kgtk --debug
/Volumes/GoogleDrive/Shared drives/KGTK-public-graphs/wikidata-20200803-v3/all.tsv.gz
/Volumes/GoogleDrive/Shared drives/KGTK-public-graphs/wikidata-20200803-v3/part.label.en.tsv.gz
/Users/pedroszekely/Downloads/kypher/temp.useful_wikidata_files_v3/wikidata.sqlite3.db


In [5]:
cd $output_path

/Users/pedroszekely/Downloads/kypher


In [6]:
!mkdir $OUT
!mkdir $TEMP

mkdir: /Users/pedroszekely/Downloads/kypher/useful_wikidata_files_v3: File exists
mkdir: /Users/pedroszekely/Downloads/kypher/temp.useful_wikidata_files_v3: File exists


Clean up the output and temp folders before we start

In [7]:
# !rm $OUT/*.tsv $OUT/*.tsv.gz
# !rm $TEMP/*.tsv $TEMP/*.tsv.gz

In [8]:
if delete_database and delete_database != "no":
    print("Deleted database")
    !rm $STORE

Uncomment the line below to remove the sqllite2 database. It takes a long time to load all the data and create indices, so don't remove the database unless you change files that have already been loaded and you need to force a reload.

### Get a sample and force importing the edge file into the database

In [7]:
!$kgtk query -i "$EDGES" --limit 10 --graph-cache $STORE

[2020-11-04 21:29:51 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     LIMIT ?
  PARAS: [10]
---------------------------------------------
id	node1	label	node2	rank	node2;wikidatatype
Q1-P1036-418bc4-78f5a565-0	Q1	P1036	"113"	normal	external-id
Q1-P1036-b98c08-1dc98be9-0	Q1	P1036	"523.1"	normal	external-id
Q1-P1051-d70eb1-60991f20-0	Q1	P1051	"517"	normal	external-id
Q1-P1245-ee25a9-46be09ed-0	Q1	P1245	"8506"	normal	external-id
Q1-P1256-8da0ce-af30f4e9-0	Q1	P1256	"51A11"	normal	external-id
Q1-P1296-f73b4e-4d0c1e5d-0	Q1	P1296	"0216407"	normal	external-id
Q1-P1343-Q19190511-ab132b87-0	Q1	P1343	Q19190511	normal	wikibase-item
Q1-P1343-Q2041543-4ed8a129-0	Q1	P1343	Q2041543	normal	wikibase-item
Q1-P1343-Q602358-12bf99e2-0	Q1	P1343	Q602358	normal	wikibase-item
Q1-P1343-Q88672152-5080b9e2-0	Q1	P1343	Q88672152	normal	wikibase-item
        0.97 real         0.57 user         0.12 sys


Force creation of the index on the label column

In [14]:
!$kgtk query -i "$EDGES" --graph-cache $STORE -o - \
    --match '(i)-[:P31]->(c)' \
    --limit 5

[2020-11-04 00:23:41 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
     LIMIT ?
  PARAS: ['P31', 5]
---------------------------------------------
[2020-11-04 00:23:41 sqlstore]: CREATE INDEX on table graph_1 column label ...
[2020-11-04 00:55:13 sqlstore]: ANALYZE INDEX on table graph_1 column label ...
id	node1	label	node2	rank	node2;wikidatatype
Q1-P31-Q36906466-ae5f57ac-0	Q1	P31	Q36906466	normal	wikibase-item
Q100-P31-Q1093829-d2e8fc52-0	Q100	P31	Q1093829	normal	wikibase-item
Q100-P31-Q1549591-56524ffd-0	Q100	P31	Q1549591	normal	wikibase-item
Q100-P31-Q21518270-a4c0505d-0	Q100	P31	Q21518270	preferred	wikibase-item
Q1000-P31-Q179023-e86c0605-0	Q1000	P31	Q179023	normal	wikibase-item
     2014.94 real      1094.60 user       460.64 sys


Force creation of the index on the node2 column

In [7]:
!$kgtk query -i "$EDGES" --graph-cache $STORE -o - \
    --match '(i)-[r]->(:Q5)' \
    --limit 5

[2020-11-04 14:23:52 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."node2"=?
     LIMIT ?
  PARAS: ['Q5', 5]
---------------------------------------------
id	node1	label	node2	rank	node2;wikidatatype
Q10000001-P31-Q5-cc1c4199-0	Q10000001	P31	Q5	normal	wikibase-item
Q1000002-P31-Q5-5c9914ea-0	Q1000002	P31	Q5	normal	wikibase-item
Q1000005-P31-Q5-4d5e2a2b-0	Q1000005	P31	Q5	normal	wikibase-item
Q1000006-P31-Q5-38728290-0	Q1000006	P31	Q5	normal	wikibase-item
Q1000015-P31-Q5-81519a6c-0	Q1000015	P31	Q5	normal	wikibase-item
        0.91 real         0.59 user         0.17 sys


### Count the number of edges

In [8]:
!$kgtk query -i "$EDGES" --graph-cache $STORE \
    --match 'all: ()-[r]->()' \
    --return 'count(r) as count' \
    --limit 10

[2020-11-04 07:08:13 query]: SQL Translation:
---------------------------------------------
  SELECT count(graph_1_c1."id") "count"
     FROM graph_1 AS graph_1_c1
     LIMIT ?
  PARAS: [10]
---------------------------------------------
count
1102827643
      632.62 real        97.92 user       153.08 sys


### Create a file that specifies for each node whether it is an item or a property

### Create the P31 and P279 files

In [9]:
!$kgtk query -i "$EDGES" --graph-cache $STORE -o $OUT/all.P31.tsv.gz \
    --match '(n1)-[l:P31]->(n2)' \
    --return 'l, n1, l.label, n2' 

[2020-11-04 07:18:46 query]: SQL Translation:
---------------------------------------------
  SELECT graph_1_c1."id", graph_1_c1."node1", graph_1_c1."label", graph_1_c1."node2"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
  PARAS: ['P31']
---------------------------------------------
     1676.81 real       940.66 user       250.77 sys


In [10]:
!$kgtk query -i "$EDGES" --graph-cache $STORE -o $OUT/all.P279.tsv.gz \
    --match '(n1)-[l:P279]->(n2)' \
    --return 'l, n1, l.label, n2' 

[2020-11-04 07:46:43 query]: SQL Translation:
---------------------------------------------
  SELECT graph_1_c1."id", graph_1_c1."node1", graph_1_c1."label", graph_1_c1."node2"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
  PARAS: ['P279']
---------------------------------------------
      106.45 real        38.45 user        18.43 sys


In [11]:
!gzcat $OUT/all.P31.tsv.gz | head | column -t -s $'\t' 

gzcat: error writing to output: Broken pipe
gzcat: /Users/pedroszekely/Downloads/kypher/useful_wikidata_files_v3/all.P31.tsv.gz: uncompress failed
id                               node1    label  node2
Q1-P31-Q36906466-ae5f57ac-0      Q1       P31    Q36906466
Q100-P31-Q1093829-d2e8fc52-0     Q100     P31    Q1093829
Q100-P31-Q1549591-56524ffd-0     Q100     P31    Q1549591
Q100-P31-Q21518270-a4c0505d-0    Q100     P31    Q21518270
Q1000-P31-Q179023-e86c0605-0     Q1000    P31    Q179023
Q1000-P31-Q3624078-3aade447-0    Q1000    P31    Q3624078
Q1000-P31-Q6256-bbaa8a03-0       Q1000    P31    Q6256
Q10000-P31-Q10876391-e428a782-0  Q10000   P31    Q10876391
Q100000-P31-Q1852859-3665587f-0  Q100000  P31    Q1852859


In [12]:
!$kgtk cat -i $OUT/all.P279.tsv.gz -i $OUT/all.P31.tsv.gz -o $OUT/all.P31_P279.tsv.gz 

     1023.85 real      1008.10 user         7.25 sys


In [13]:
!gzcat $OUT/all.P31_P279.tsv | head | column -t -s $'\t' 

gzcat: id                                  node1     label  node2
error writing to outputQ1000032-P279-Q1813494-0aa0f1dc-0   Q1000032  P279   Q1813494
: Q1000032-P279-Q83602-482a1943-0     Q1000032  P279   Q83602
Broken pipe
Q1000039-P279-Q11555767-2dddfd86-0  Q1000039  P279   Q11555767
Q1000064-P279-Q11016-0ab23344-0     Q1000064  P279   Q11016
Q1000084-P279-Q159810-7852671b-0    Q1000084  P279   Q159810
Q1000108-P279-Q849640-02ec6f84-0    Q1000108  P279   Q849640
Q1000116-P279-Q179692-f8d04cc2-0    Q1000116  P279   Q179692
Q1000120-P279-Q12510-3e6da659-0     Q1000120  P279   Q12510
gzcat: Q1000120-P279-Q181296-f30f7109-0    Q1000120  P279   Q181296
/Users/pedroszekely/Downloads/kypher/useful_wikidata_files_v3/all.P31_P279.tsv.gz: uncompress failed


### Create the file that contains all nodes reachable via P279 starting from a node2 in P31 or a node1 in P279

First compute the roots

In [24]:
!$kgtk query -i $OUT/all.P279.tsv.gz --graph-cache $STORE -o $TEMP/P279.n1.tsv.gz \
    --match '(n1)-[l]->()' \
    --return 'n1 as id' 

[2020-11-04 20:51:51 query]: SQL Translation:
---------------------------------------------
  SELECT graph_2_c1."node1" "id"
     FROM graph_2 AS graph_2_c1
  PARAS: []
---------------------------------------------
       16.80 real        15.43 user         0.57 sys


In [25]:
!$kgtk query -i $OUT/all.P31.tsv.gz --graph-cache $STORE  -o $TEMP/P31.n2.tsv.gz \
    --match '()-[l]->(n2)' \
    --return 'n2 as id' 

[2020-11-04 20:52:07 query]: SQL Translation:
---------------------------------------------
  SELECT graph_3_c1."node2" "id"
     FROM graph_3 AS graph_3_c1
  PARAS: []
---------------------------------------------
      175.55 real       154.56 user        11.03 sys


In [29]:
!$kgtk cat --mode NONE -i $TEMP/P31.n2.tsv.gz $TEMP/P279.n1.tsv.gz \
    | gzip > $TEMP/P279.roots.1.tsv.gz

      348.65 real       346.10 user         1.47 sys


In [8]:
!$kgtk sort2 --mode NONE --column id -i $TEMP/P279.roots.1.tsv.gz \
    | gzip > $TEMP/P279.roots.2.tsv.gz

       70.85 real        56.25 user        12.59 sys


In [9]:
!zcat < $TEMP/P279.roots.2.tsv.gz | head

id
Q1
Q1
Q1000032
Q1000032
Q1000039
Q1000064
Q1000084
Q1000108
Q1000116
zcat: error writing to output: Broken pipe


In [10]:
!$kgtk compact -i $TEMP/P279.roots.2.tsv.gz --mode NONE \
    --presorted \
    --columns id \
> $TEMP/P279.roots.tsv

      382.41 real       380.34 user         1.00 sys


Now we can invoke the reachable-nodes command

In [13]:
!$kgtk reachable-nodes \
    --rootfile $TEMP/P279.roots.tsv \
    --selflink \
    -i $OUT/all.P279.tsv.gz \
    | gzip > $TEMP/P279.reachable.tsv.gz

     5839.30 real      3373.77 user      2374.71 sys


In [14]:
!zcat < $TEMP/P279.reachable.tsv.gz | head

node1	label	node2
Q1000032	reachable	Q1000032
Q1000032	reachable	Q1813494
Q1000032	reachable	Q1799072
Q1000032	reachable	Q16686448
Q1000032	reachable	Q35120
Q1000032	reachable	novalue
Q1000032	reachable	Q2695280
Q1000032	reachable	Q1914636
Q1000032	reachable	Q20937557
zcat: error writing to output: Broken pipe


The reachable-nodes command produces edges labeled `reachable`, so we need one command to rename them.

In [15]:
!$kgtk query -i $TEMP/P279.reachable.tsv.gz --graph-cache $STORE  -o $TEMP/P279star.1.tsv.gz \
    --match '(n1)-[]->(n2)' \
    --return 'n1, "P279star" as label, n2 as node2' 

[2020-11-05 09:25:49 sqlstore]: DROP graph data table graph_4 from /Users/pedroszekely/Downloads/kypher/temp.useful_wikidata_files_v3/P279.reachable.tsv.gz
[2020-11-05 09:25:49 sqlstore]: IMPORT graph directly into table graph_4 from /Users/pedroszekely/Downloads/kypher/temp.useful_wikidata_files_v3/P279.reachable.tsv.gz ...
[2020-11-05 09:29:12 query]: SQL Translation:
---------------------------------------------
  SELECT graph_4_c1."node1", ? "label", graph_4_c1."node2" "node2"
     FROM graph_4 AS graph_4_c1
  PARAS: ['P279star']
---------------------------------------------
      847.92 real       955.62 user        28.23 sys


Now we can concatenate these files to produce the final output

In [19]:
!$kgtk sort2 -i $TEMP/P279star.1.tsv.gz -o $TEMP/P279star.2.tsv.gz

      284.59 real       242.05 user        65.20 sys


In [20]:
!$kgtk compact --presorted -i $TEMP/P279star.2.tsv.gz -o $TEMP/P279star.3.tsv.gz

     1371.31 real      1362.32 user         3.93 sys


In [28]:
!$kgtk add-id --id-style node1-label-node2-num -i $TEMP/P279star.3.tsv.gz -o $OUT/all.P279star.tsv.gz

     1336.10 real      1284.93 user        28.65 sys


In [29]:
!zcat < $OUT/all.P279star.tsv.gz | head

node1	label	node2	id
Q1000032	P279star	Q1000032	Q1000032-P279star-Q1000032-0000
Q1000032	P279star	Q1150070	Q1000032-P279star-Q1150070-0000
Q1000032	P279star	Q1190554	Q1000032-P279star-Q1190554-0000
Q1000032	P279star	Q133500	Q1000032-P279star-Q133500-0000
Q1000032	P279star	Q13878858	Q1000032-P279star-Q13878858-0000
Q1000032	P279star	Q14819853	Q1000032-P279star-Q14819853-0000
Q1000032	P279star	Q14912053	Q1000032-P279star-Q14912053-0000
Q1000032	P279star	Q16686448	Q1000032-P279star-Q16686448-0000
Q1000032	P279star	Q16722960	Q1000032-P279star-Q16722960-0000
zcat: error writing to output: Broken pipe


This is how we would do the typical `?item P31/P279* ?class` in Kypher. 
The example shows how to get all the `n1` that are instances of subclasses of beer (q44).

In [56]:
!$kgtk query -i $OUT/all.P31.tsv.gz -i $OUT/all.P279star.tsv.gz -i "$LABELS" --graph-cache $STORE  \
--match 'P31: (n1)-[:P31]->(c), P279star: (c)-[]->(:Q44), label: (n1)-[:label]->(label), label: (c)-[:label]->(c_label)' \
--return 'distinct n1, c as class, count(c) as count, label as n1_label, c_label as `class name`' \
--order-by 'n1, c desc, count(n1) desc' \
--limit 10 \
| column -t -s $'\t' 

[2020-11-05 19:26:57 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_3_c1."node1", graph_5_c2."node1" "class", count(graph_5_c2."node1") "count", graph_7_c3."node2" "n1_label", graph_7_c4."node2" "class name"
     FROM graph_3 AS graph_3_c1, graph_5 AS graph_5_c2, graph_7 AS graph_7_c3, graph_7 AS graph_7_c4
     WHERE graph_3_c1."label"=?
     AND graph_5_c2."node2"=?
     AND graph_7_c3."label"=?
     AND graph_7_c4."label"=?
     AND graph_3_c1."node1"=graph_7_c3."node1"
     AND graph_3_c1."node2"=graph_5_c2."node1"
     AND graph_3_c1."node2"=graph_7_c4."node1"
     GROUP BY graph_3_c1."node1", class
     ORDER BY graph_3_c1."node1" ASC, graph_5_c2."node1" DESC, count(graph_3_c1."node1") DESC
     LIMIT ?
  PARAS: ['P31', 'Q44', 'label', 'label', 10]
---------------------------------------------
        1.84 real         0.65 user         0.35 sys
node1      class      count  n1_label           class name
Q1000737   Q15075508  1      

In [59]:
!$kgtk query -i $OUT/all.P279star.tsv.gz -i "$LABELS" --graph-cache $STORE  \
--match 'star: (n1)-[]->(c:Q44)' \
--limit 10 \
| column -t -s $'\t' 

[2020-11-05 19:29:23 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_5 AS graph_5_c1
     WHERE graph_5_c1."node2"=?
     LIMIT ?
  PARAS: ['Q44', 10]
---------------------------------------------
        0.77 real         0.61 user         0.14 sys
node1      label     node2  id
Q10262008  P279star  Q44    Q10262008-P279star-Q44-0000
Q10313616  P279star  Q44    Q10313616-P279star-Q44-0000
Q10314719  P279star  Q44    Q10314719-P279star-Q44-0000
Q10317863  P279star  Q44    Q10317863-P279star-Q44-0000
Q10340754  P279star  Q44    Q10340754-P279star-Q44-0000
Q10350781  P279star  Q44    Q10350781-P279star-Q44-0000
Q10355535  P279star  Q44    Q10355535-P279star-Q44-0000
Q10382024  P279star  Q44    Q10382024-P279star-Q44-0000
Q10461865  P279star  Q44    Q10461865-P279star-Q44-0000
Q1046968   P279star  Q44    Q1046968-P279star-Q44-0000


### Create a file to do generalized Is-A queries
The idea is that `(n1)-[:isa]->(n2)` when `(n1)-[:P31]->(n2)` or `(n1)-[:P279]->(n2)`

We do this by concatenating the files and renaming the relation

In [31]:
!$kgtk cat -i $OUT/all.P31.tsv.gz $OUT/all.P279.tsv.gz \
    | gzip > $TEMP/isa.1.tsv.gz

      538.04 real       519.06 user         9.69 sys


In [32]:
!$kgtk query -i $TEMP/isa.1.tsv.gz --graph-cache $STORE  -o $OUT/all.isa.tsv.gz \
    --match '(n1)-[]->(n2)' \
    --return 'n1, "isa" as label, n2' 

[2020-11-05 14:26:22 sqlstore]: IMPORT graph directly into table graph_6 from /Users/pedroszekely/Downloads/kypher/temp.useful_wikidata_files_v3/isa.1.tsv.gz ...
[2020-11-05 14:33:10 query]: SQL Translation:
---------------------------------------------
  SELECT graph_6_c1."node1", ? "label", graph_6_c1."node2"
     FROM graph_6 AS graph_6_c1
  PARAS: ['isa']
---------------------------------------------
      835.15 real      1061.27 user        43.66 sys


Example of how to use the `isa` relation

In [62]:
!$kgtk query -i $OUT/all.isa.tsv.gz -i $OUT/all.P279star.tsv.gz -i "$LABELS" --graph-cache $STORE  -o - \
    --match 'isa: (n1)-[l:isa]->(c), P279star: (c)-[]->(:Q44), label: (n1)-[:label]->(label)' \
    --return 'distinct n1, l.label, "Q44" as node2, label as n1_label' \
    --limit 10

[2020-11-05 20:07:32 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_8_c1."node1", graph_8_c1."label", ? "node2", graph_7_c3."node2" "n1_label"
     FROM graph_5 AS graph_5_c2, graph_7 AS graph_7_c3, graph_8 AS graph_8_c1
     WHERE graph_5_c2."node2"=?
     AND graph_7_c3."label"=?
     AND graph_8_c1."label"=?
     AND graph_5_c2."node1"=graph_8_c1."node2"
     AND graph_7_c3."node1"=graph_8_c1."node1"
     LIMIT ?
  PARAS: ['Q44', 'Q44', 'label', 'isa', 10]
---------------------------------------------
[2020-11-05 20:07:32 sqlstore]: CREATE INDEX on table graph_8 column node1 ...
[2020-11-05 20:08:27 sqlstore]: ANALYZE INDEX on table graph_8 column node1 ...
node1	label	node2	n1_label
Q15875298	isa	Q44	'Floreffe'@en
Q1917255	isa	Q44	'St-Idesbald'@en
Q2004062	isa	Q44	'Sancti Adalberti'@en
Q2006077	isa	Q44	'Bonne-Espérance abbey'@en
Q2272636	isa	Q44	'Ename beer'@en
Q2290730	isa	Q44	'Ter Dolen (beer)'@en
Q3625571	isa	Q44	'Herkenrode Tripel

### Creating a subset of Wikidata without scholarly articles (Q13442814)
First create a file with the schloarly articles

In [71]:
!$kgtk query -i $OUT/all.isa.tsv.gz -i $OUT/all.P279star.tsv.gz --graph-cache $STORE  -o $OUT/all.isa.Q13442814.Q523.Q16521.Q318.Q7318358.Q7187.Q11173.Q8054.tsv.gz \
--match 'isa: (n1)-[l:isa]->(class)' \
--where 'class in ["Q13442814", "Q523", "Q16521", "Q318", "Q7318358", "Q7187", "Q11173", "Q8054"]' \
--return 'distinct n1, l.label, class as node2' 

[2020-11-06 10:01:39 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_8_c1."node1", graph_8_c1."label", graph_8_c1."node2" "node2"
     FROM graph_8 AS graph_8_c1
     WHERE graph_8_c1."label"=?
     AND (graph_8_c1."node2" IN (?, ?, ?, ?, ?, ?, ?, ?))
  PARAS: ['isa', 'Q13442814', 'Q523', 'Q16521', 'Q318', 'Q7318358', 'Q7187', 'Q11173', 'Q8054']
---------------------------------------------
      411.40 real       279.05 user        46.43 sys


In [ ]:
!$kypher -i $NKG \
--match '(n1)-[r {label: label}]->(n2), (r)-[:source]->(source)' \
--where 'source = $s and n1 =~ ".*/catch/.*"' \
--return 'distinct n1 as node1' \
--spara s='CN' \
--limit 100

In [65]:
!gzcat $OUT/all.isa.Q13442814.tsv | wc

 35933570 107800710 862405612


Now we need to remove from `$EDGES` any edge where node1 or node2 is in node1 of `$OUT/all.isa.Q13442814.tsv`. The result will be `$OUT/minus.Q13442814.tsv`. We can then run the whole notebook with this new file as $EDGES and compute all the product files in a new output directory

In [68]:
!$kgtk query -i $OUT/all.isa.Q13442814.tsv.gz -i "$EDGES" --graph-cache $STORE  \
--match 'isa: (n1)-[]->(), all: (n2)-[l]->(n3)' \
--where 'n1 != n2 or n1 != n3' \
--return 'distinct n2 as node1, l.label as label, n3 as node2, l as id' \
-o $OUT/all.minus.Q13442814.tsv.gz

[2020-11-05 20:45:35 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c2."node1" "node1", graph_1_c2."label" "label", graph_1_c2."node2" "node2", graph_1_c2."id" "id"
     FROM graph_1 AS graph_1_c2, graph_9 AS graph_9_c1
     WHERE ((graph_9_c1."node1" != graph_1_c2."node1") OR (graph_9_c1."node1" != graph_1_c2."node2"))
  PARAS: []
---------------------------------------------
^C


## Summary

In [47]:
!wc -l $OUT/*.tsv $OUT/*.tsv.gz $EDGES

    7479 /Users/pedroszekely/Downloads/kypher/useful_wikidata_files_2/all-distribution.tsv
   45941 /Users/pedroszekely/Downloads/kypher/useful_wikidata_files_2/all.P279.tsv.gz
       0 /Users/pedroszekely/Downloads/kypher/useful_wikidata_files_2/all.P279star.tsv.gz
 2206506 /Users/pedroszekely/Downloads/kypher/useful_wikidata_files_2/all.P31.tsv.gz
 2254104 /Users/pedroszekely/Downloads/kypher/useful_wikidata_files_2/all.P31_P279.tsv.gz
       0 /Users/pedroszekely/Downloads/kypher/useful_wikidata_files_2/all.isa.Q13442814.tsv.gz
 1208961 /Users/pedroszekely/Downloads/kypher/useful_wikidata_files_2/all.isa.tsv.gz
       0 /Users/pedroszekely/Downloads/kypher/useful_wikidata_files_2/all.wikidatatype.distribution.tsv.gz
  374344 /Users/pedroszekely/Downloads/kypher/useful_wikidata_files_2/part.alias.en.tsv.gz
  383165 /Users/pedroszekely/Downloads/kypher/useful_wikidata_files_2/part.alias.tsv.gz
       0 /Users/pedroszekely/Downloads/kypher/useful_wikidata_files_2/part.commonsMedia.tsv.

Number of distinct items in our dataset

In [48]:
!$kgtk query -i $EDGES --graph-cache $STORE  -o - \
    --match '(n1)-[]->()' \
    --return 'count(distinct n1) as count'

[2020-10-22 02:52:30 query]: SQL Translation:
---------------------------------------------
  SELECT count(DISTINCT graph_1_c1."node1") "count"
     FROM graph_1 AS graph_1_c1
  PARAS: []
---------------------------------------------
count
88228944
     1364.75 real      1000.96 user       122.75 sys
